# Source

## Define NetSuite REST connnection

In [ ]:
import os
import requests
import oauthlib.oauth1

consumer_key = dbutils.secrets.get(scope="azure_key_vault", key="NS-CONSUMER-KEY") # use Azure Key Vault to save this. 
consumer_secret = dbutils.secrets.get(scope="azure_key_vault", key="NS-CONSUMER-SECRET") # use Azure Key Vault to save this. 
access_token = dbutils.secrets.get(scope="azure_key_vault", key="NS-TOKEN-ID") # use Azure Key Vault to save this. 
token_secret = dbutils.secrets.get(scope="azure_key_vault", key="NS-TOKEN-SECRET") # use Azure Key Vault to save this. 
realm = os.getenv("NS_REALM") # change to NS_REALM in production.

oauth = oauthlib.oauth1.Client(
    client_key=consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=token_secret,
    signature_method='HMAC-SHA256',
    signature_type='AUTH_HEADER',
    realm=realm
)

def suiteQL(oauth, url, body):
    uri, headers, _ = oauth.sign(url, http_method='POST')
    headers['Prefer'] = 'transient' # SuiteQL requires this specific header
    return requests.post(uri, data=body, headers=headers)

def queryAllCompanies(oauth, url, body):
    customers = []
    while url:
        json_response = suiteQL(oauth, url, body).json()
        customers.extend(json_response['items']) # extend will merge two lists. append adds a single element, while extend concatenates another list.
        next_link = next((link for link in json_response['links'] if link['rel'] == 'next'), None) # The next() function in Python is used to retrieve the next item from an iterator, json_response['links']. The None in this context for the next() call serves as a default return value if no next link is found.
        url = next_link['href'] if next_link else None # if no next_link, stop the while loop.
    return customers

## Query B2B customers by NetSuite SuiteQL

SuiteQL does not have address information.

B2B customers are in salesRep = 227. Ingore the customers that do not have name or email.

In [ ]:
# The initial URL you want to start
offset = 0
url = "https://{0}.suitetalk.api.netsuite.com/services/rest/query/v1/suiteql?limit=1000&offset={1}".format(os.getenv("NS_ACCOUNT_ID"), offset)

# create body.
body = '''
{
    "q": "SELECT DISTINCT c.id, c.companyName, c.email, c.custentity3, c.salesRep, ea.addr1, ea.addr2, ea.addr3, ea.city, ea.state, ea.zip, ea.country, ea.addressee FROM Customer c LEFT JOIN EntityAddressbook eab ON eab.entity = c.id LEFT JOIN EntityAddress ea ON ea.nkey = eab.AddressBookAddress WHERE salesRep = 227 AND companyName IS NOT NULL AND eab.defaultbilling = 'T' AND c.IsInactive = 'F'"
}
'''

# query all B2B customers
customers = queryAllCompanies(oauth, url, body)

In [ ]:
len(customers)

2766

# Destination

## Define Shopify GraphQL connection

In [ ]:
import shopify

# Define Shopify B2B private API connection
shop = os.getenv("SHOPIFY_B2B")
shop_url = f'{shop}.myshopify.com'
api_version = "2023-04" # the lastest version that can be supported by ShopifyAPI.
private_app_password = dbutils.secrets.get(scope="azure_key_vault", key="SHOPIFYB2B-PW") # use Azure Key Vault to save this password. 

# Create a Shopify session
session = shopify.Session(shop_url, api_version, private_app_password)

## Create new companies and related customers on Shopify B2B

If the company exists in Shopify B2B, the result will throw an error: TAKEN and won't create a new one on Shopify B2B. If the company does not exist, it will create the company and related customer on Shopify B2B.

Please note that a company can only link to a customer that is not taken by other companies.

NetSuite does not have the main contact's lastName and firstName. However, email and firstName are required on Shopify. Hence, firstName defaults as B2B and use companyName as lastName. email should not use the one from NetSuite. Instead, create a different email for each company following this pattern: [companyname@b2b.com].

Ignore zoneCode because NetSuite may not have the two-letter code for it. [https://shopify.dev/docs/api/admin-graphql/2024-01/input-objects/CompanyAddressInput](https://shopify.dev/docs/api/admin-graphql/2024-01/input-objects/CompanyAddressInput)

In [ ]:
# Active the shopify session
shopify.ShopifyResource.activate_session(session)

# Execute GraphQL query and iterate over the customers
for customer in customers[:2]: # Take only first 2 customers. Remove it in PRODUCTION.
    companyname = customer.get('companyname')
    address1 = customer.get('addr1')
    address2 = ("" if not customer.get('addr2') else customer.get('addr2'))
    city = customer.get('city')
    zoneCode = customer.get('state')
    zip = customer.get('zip')
    countryCode = customer.get('country')
    recipient = customer.get('addressee')
    email = customer.get('companyname').replace(' ', '').lower() + '@b2b.com'
    query = '''
    mutation MyMutation {
        companyCreate(
            input: {company: {name: "%s"}, companyContact: {email: "%s", lastName: "%s", firstName: "B2B"}, companyLocation: {billingAddress: {address1: "%s", address2: "%s", city: "%s", zip: "%s", countryCode: %s, recipient: "%s"}, , shippingAddress: {address1: "%s", address2: "%s", city: "%s", zip: "%s", countryCode: %s, recipient: "%s"}}}
        ) {
            userErrors {
                code
                field
            }
        }
    }
    ''' % (companyname, email, companyname, address1, address2, city, zip, countryCode, recipient, address1, address2, city, zip, countryCode, recipient)  # Use string interpolation to insert the values into the query
    result = shopify.GraphQL().execute(query)

# Disactivate the shopify session
shopify.ShopifyResource.clear_session()

In [ ]:
result

'{"data":{"companyCreate":{"userErrors":[]}},"extensions":{"cost":{"requestedQueryCost":10,"actualQueryCost":10,"throttleStatus":{"maximumAvailable":20000.0,"currentlyAvailable":19990,"restoreRate":1000.0}}}}'